In [1]:
import pandas as pd

def get_rank(file_name, asc=False):
    data = pd.read_csv(file_name, names=['id','w'])
    data['rank'] = data['w'].rank(
        method='min', ascending=asc).apply(int)
    # return data.sort_values(by=['rank'])
    result = {}
    def f(row):
        result[row['id']] = row['rank']
    data.apply(f, axis=1)

    # for i, row in data.iterrows():
    #     result[row['id']] = row['rank']
    return result

# get_rank('true_rank.txt', True)
# get_rank('result_pagerank_original_indirected.txt')
# get_rank('result_pagerank_weighted_indirected.txt')
# get_rank('k_shell.txt')

# true = pd.read_csv('true_rank.txt', names=['id','rank']).sort_values(by=['rank'])
# true.loc[true['rank'] <= 100]


In [2]:
from pygrank import Graph
# from pagerank3 import extract_from_file

# info = extract_from_file('amazon-meta.txt')
def import_graph(file_path):
    graph = Graph()

    last_ASIN = None
    with open(file_path, 'r') as f:
        while True:
            line = f.readline()

            if line == '':
                break

            if line.startswith('ASIN: '):
                last_ASIN = line.strip().split()[1]
            elif line.strip().startswith('similar'):
                similar = line.strip().split()[2:]
                for s in similar:
                    graph.add_edge(last_ASIN, s)
    return graph

graph = import_graph('amazon-meta.txt')

len(graph)

The default pygrank backend has been set to "numpy" by the file /home/hung/.pygrank/config.json
Set your preferred backend as one of ["numpy", "pytorch", "tensorflow", "torch_sparse", "matvec", "sparse_dot_mkl"] and "reminder": false in that file to remove this message from future runs.


554789

In [3]:
import pygrank as pg


ranks = get_rank('result_pagerank_weighted_indirected.txt')
ground = get_rank('true_rank.txt', True)

to_remove = [i for i in ranks if i not in ground]
for i in to_remove:
    del ranks[i]

sub_graph = graph.subgraph(ground)

signal = pg.to_signal(sub_graph, ranks)
true = pg.to_signal(sub_graph, ground)

tpr = pg.TPR(known_scores=true).evaluate(signal) 
auc = pg.Density().evaluate(signal)
ndcg = pg.NDCG(k=1000, known_scores=ground).evaluate(signal)
print(f'TPR: {tpr}\nAUC: {auc} \nNDCG@1000: {ndcg}')

TPR: 0.3361604678763454
AUC: 1.2711112405638428e-05 
NDCG@1000: 0.6259148437777695


In [4]:
import pygrank as pg


ranks = get_rank('k_shell.txt')
ground = get_rank('true_rank.txt', True)

to_remove = [i for i in ranks if i not in ground]
for i in to_remove:
    del ranks[i]

sub_graph = graph.subgraph(ground)

signal = pg.to_signal(sub_graph, ranks)
true = pg.to_signal(sub_graph, ground)

tpr = pg.TPR(known_scores=true).evaluate(signal) 
auc = pg.Density().evaluate(signal)
ndcg = pg.NDCG(k=1000, known_scores=ground).evaluate(signal)
print(f'TPR: {tpr}\nAUC: {auc} \nNDCG@1000: {ndcg}')


TPR: 0.2159747731496687
AUC: 1.1096675329550079e-05 
NDCG@1000: 0.5503147715898823
